In [1]:
!pip -qq install transformers

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 2.9MB 52.7MB/s 
     |████████████████████████████████| 890kB 58.0MB/s 


In [2]:
!git clone https://github.com/Ryzhtus/master-thesis

Cloning into 'master-thesis'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 38 (delta 4), reused 38 (delta 4), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [3]:
cd master-thesis

/content/master-thesis


In [4]:
from named_entity_recognition.dataset import ConLL2003Dataset, read_data, create_dataset_and_dataloader
from named_entity_recognition.model import BertNER
from named_entity_recognition.train import train_model, test 

from transformers import BertTokenizer
from transformers import AdamW
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
DEVICE = 'cuda' if torch.cuda.is_available else 'cpu'
EPOCHS = 4
BATCH_SIZE = 16

In [5]:
sentences, tags, tags_number = read_data("/content/master-thesis/data/conll2003/train.txt")
dataset = ConLL2003Dataset(sentences, tags, tags_number, TOKENIZER)

In [6]:
train_dataset, train_dataloader = create_dataset_and_dataloader("/content/master-thesis/data/conll2003/train.txt", BATCH_SIZE, TOKENIZER)
eval_dataset, eval_dataloader = create_dataset_and_dataloader("/content/master-thesis/data/conll2003/valid.txt", BATCH_SIZE, TOKENIZER)
test_dataset, test_dataloader = create_dataset_and_dataloader("/content/master-thesis/data/conll2003/test.txt", BATCH_SIZE, TOKENIZER)

In [7]:
classes = len(dataset.ner_tags)

model = BertNER(classes).to(DEVICE)

optimizer = AdamW(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(DEVICE)

train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, train_dataset.tag2idx, DEVICE, EPOCHS)


Epoch 1 / 4
Train Loss = 0.48401, F1-score = 87.364%
Eval Loss  = 0.36696, F1-score = 89.795%
Epoch 2 / 4
Train Loss = 0.34209, F1-score = 90.321%
Eval Loss  = 0.33238, F1-score = 90.784%
Epoch 3 / 4
Train Loss = 0.30447, F1-score = 91.111%
Eval Loss  = 0.33240, F1-score = 90.542%
Epoch 4 / 4
Train Loss = 0.27080, F1-score = 91.963%
Eval Loss  = 0.28407, F1-score = 91.573%


In [8]:
test(model, criterion, test_dataloader, train_dataset.tag2idx, DEVICE)

Test Loss  = 0.71206, F1-score = 85.320%
